In [1]:
import polars as pl
import pathlib
import pint

In [2]:
cwd = pathlib.Path.cwd()

if cwd.name == 'Mild-Steel-Tempering':
    print("Path is project root")
else:
    print("Please correct current working directory to the project root")

Path is project root


In [3]:
resources_path = pathlib.PurePath(pathlib.PurePath(cwd), 'resources')
resources_path

PurePosixPath('/home/mox/Documents/coding_projects/bootcamp_local/Homeworks/Mild-Steel-Tempering/resources')

In [4]:
data_path = f"{resources_path}/scraped_properties.csv"
df_data = pl.read_csv(data_path)
df_data.glimpse()

Rows: 28
Columns: 6
$ steel           <str> 'AISI 6145', 'AISI-SAE 4027', 'AISI 1090', 'AISI 4640', 'AISI-SAE 1080', 'AISI 4047', 'AISI-SAE 1030', 'AISI-SAE 5140', 'AISI 1095', 'AISI-SAE 1050'
$ density         <str> '7.75', '7.85', '7.85', '7.75', '7.7-8.03', '7.85', '7.85', '7.85', '7.85', '7.85'
$ units_density   <str> 'g/cm3', 'g/cm3', 'g/cm3', 'g/cm3', 'g/cm3', 'g/cm3', 'g/cc', 'g/cm3', 'g/cm3', 'g/cm3'
$ poissons_ratio  <str> '0.27-030', '0.27-0.30', '0.27-0.30', '0.27-030', '0.27-0.30', '0.27-0.30', '0.27-0.30', '0.27-0.30', '0.27-0.30', '0.27-0.30'
$ elastic_modulus <f64> 7.85, 7.85, 7.85, 7.85, 7.85, 7.85, 7.85, 7.85, 7.85, 7.85
$ units_em        <str> 'g/cm3', 'g/cm3', 'g/cm3', 'g/cm3', 'g/cm3', 'g/cm3', 'g/cm3', 'g/cm3', 'g/cm3', 'g/cm3'

